# Train Tensorflow Model Distributed on Batch AI
In this notebook we will train a TensorFlow model ([ResNet50](https://arxiv.org/abs/1512.03385)) in a distributed fashion using [Horovod](https://github.com/uber/horovod) on the Imagenet dataset. This tutorial will take you through the following steps:
 * [Create Azure Resources](#azure_resources)
 * [Create Fileserver(NFS)](#create_fileshare)
 * [Configure Batch AI Cluster](#configure_cluster)
 * [Submit and Monitor Job](#job)
 * [Clean Up Resources](#clean_up)

In [2]:
import sys
sys.path.append("../common") 

from dotenv import dotenv_values, set_key, find_dotenv, get_key
from getpass import getpass
import os
import json
from utils import get_password, write_json_to_file

Below are the variables that describe our experiment. By default we are using the NC24rs_v3 (Standard_NC24rs_v3) VMs which have V100 GPUs and Infiniband. By default we are using 2 nodes with each node having 4 GPUs, this equates to 8 GPUs. Feel free to increase the number of nodes but be aware what limitations your subscription may have.

Set the USE_FAKE to True if you want to use fake data rather than the Imagenet dataset. This is often a good way to debug your models as well as checking what IO overhead is.

In [ ]:
# Variables for Batch AI - change as necessary
ID                     = "ddtf2"
GROUP_NAME             = f"batch{ID}rg"
STORAGE_ACCOUNT_NAME   = f"batch{ID}st"
FILE_SHARE_NAME        = f"batch{ID}share"
SELECTED_SUBSCRIPTION  = "Team Danielle Internal" #"<YOUR SUBSCRIPTION>"
WORKSPACE              = "workspace"
NUM_NODES              = 2
CLUSTER_NAME           = "msv100"
VM_SIZE                = "Standard_NC24rs_v3"
GPU_TYPE               = "V100"
PROCESSES_PER_NODE     = 4
LOCATION               = "eastus"
NFS_NAME               = f"batch{ID}nfs"
EXPERIMENT             = f"distributed_tensorflow_{GPU_TYPE}"
USERNAME               = "batchai_user"
USE_FAKE               = False
DOCKERHUB              = "caia" #"<YOUR DOCKERHUB>"

In [ ]:
FAKE='-env FAKE=True' if USE_FAKE else ''
TOTAL_PROCESSES = PROCESSES_PER_NODE * NUM_NODES

<a id='azure_resources'></a>
## Create Azure Resources
First we need to log in to our Azure account. 

In [ ]:
!az login -o table

If you have more than one Azure account you will need to select it with the command below. If you only have one account you can skip this step.

In [ ]:
!az account set --subscription "$SELECTED_SUBSCRIPTION"

In [ ]:
!az account list -o table

Next we create the group that will hold all our Azure resources.

In [ ]:
!az group create -n $GROUP_NAME -l $LOCATION -o table

We will create the storage account that will store our fileshare where all the outputs from the jobs will be stored.

In [ ]:
json_data = !az storage account create -l $LOCATION -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME --sku Standard_LRS
print('Storage account {} provisioning state: {}'.format(STORAGE_ACCOUNT_NAME, 
                                                         json.loads(''.join(json_data))['provisioningState']))

In [ ]:
json_data = !az storage account keys list -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME
storage_account_key = json.loads(''.join([i for i in json_data if 'WARNING' not in i]))[0]['value']

In [ ]:
!az storage share create --account-name $STORAGE_ACCOUNT_NAME \
--account-key $storage_account_key --name $FILE_SHARE_NAME

In [ ]:
!az storage directory create --share-name $FILE_SHARE_NAME  --name scripts \
--account-name $STORAGE_ACCOUNT_NAME --account-key $storage_account_key

Here we are setting some defaults so we don't have to keep adding them to every command

In [ ]:
!az configure --defaults location=$LOCATION
!az configure --defaults group=$GROUP_NAME

In [ ]:
%env AZURE_STORAGE_ACCOUNT $STORAGE_ACCOUNT_NAME
%env AZURE_STORAGE_KEY=$storage_account_key

#### Create Workspace
Batch AI has the concept of workspaces and experiments. Below we will create the workspace for our work.

In [ ]:
!az batchai workspace create -n $WORKSPACE -g $GROUP_NAME

<a id='create_fileshare'></a>
## Create Fileserver
In this example we will store the data on an NFS fileshare. It is possible to use many storage solutions with Batch AI. NFS offers the best traideoff between performance and ease of use. The best performance is achieved by loading the data locally but this can be cumbersome since it requires that the data is download by the all the nodes which with the imagenet dataset can take hours. 

In [ ]:
!az batchai file-server create -n $NFS_NAME --disk-count 4 --disk-size 250 -w $WORKSPACE \
-s Standard_DS4_v2 -u $USERNAME -p {get_password(find_dotenv())} -g $GROUP_NAME --storage-sku Premium_LRS

In [ ]:
!az batchai file-server list -o table -w $WORKSPACE -g $GROUP_NAME

In [ ]:
json_data = !az batchai file-server list -w $WORKSPACE -g $GROUP_NAME
nfs_ip=json.loads(''.join([i for i in json_data if 'WARNING' not in i]))[0]['mountSettings']['fileServerPublicIp']

After we have created the NFS share we need to copy the data to it. To do this we write the script below which will be executed on the fileserver. It installs a tool called azcopy and then downloads and extracts the data to the appropriate directory.

In [ ]:
%%writefile nodeprep.sh
#!/usr/bin/env bash
wget https://gist.githubusercontent.com/msalvaris/073c28a9993d58498957294d20d74202/raw/87a78275879f7c9bb8d6fb9de8a2d2996bb66c24/install_azcopy
chmod 777 install_azcopy
sudo ./install_azcopy

mkdir -p /data/imagenet
azcopy --source https://datasharesa.blob.core.windows.net/imagenet/validation.csv \
        --destination  /data/imagenet/validation.csv\
        --source-sas "?se=2025-01-01&sp=r&sv=2017-04-17&sr=b&sig=7x3rN7c/nlXbnZ0gAFywd5Er3r6MdwCq97Vwvda25WE%3D"\
        --quiet

azcopy --source https://datasharesa.blob.core.windows.net/imagenet/validation.tar.gz \
        --destination  /data/imagenet/validation.tar.gz\
        --source-sas "?se=2025-01-01&sp=r&sv=2017-04-17&sr=b&sig=zy8L4shZa3XXBe152hPnhXsyfBqCufDOz01a9ZHWU28%3D"\
        --quiet

azcopy --source https://datasharesa.blob.core.windows.net/imagenet/train.csv \
        --destination  /data/imagenet/train.csv\
        --source-sas "?se=2025-01-01&sp=r&sv=2017-04-17&sr=b&sig=EUcahDDZcefOKtHoVWDh7voAC1BoxYNM512spFmjmDU%3D"\
        --quiet

azcopy --source https://datasharesa.blob.core.windows.net/imagenet/train.tar.gz \
        --destination  /data/imagenet/train.tar.gz\
        --source-sas "?se=2025-01-01&sp=r&sv=2017-04-17&sr=b&sig=qP%2B7lQuFKHo5UhQKpHcKt6p5fHT21lPaLz1O/vv4FNU%3D"\
        --quiet

cd /data/imagenet
tar -xzf train.tar.gz
tar -xzf validation.tar.gz

Next we will copy the file over and run it on the NFS VM. This will install azcopy and download and prepare the data

In [ ]:
!sshpass -p {get_password(dotenv_path)} scp -o "StrictHostKeyChecking=no" nodeprep.sh $USERNAME@{nfs_ip}:~/

In [ ]:
!sshpass -p {get_password(dotenv_path)} ssh -o "StrictHostKeyChecking=no" $USERNAME@{nfs_ip} "sudo chmod 777 ~/nodeprep.sh && ./nodeprep.sh"

Next we create our experiment.

In [ ]:
!az batchai experiment create -n $EXPERIMENT -g $GROUP_NAME -w $WORKSPACE

<a id='configure_cluster'></a>
## Configure Batch AI Cluster
We then upload the scripts we wish to execute onto the fileshare. The fileshare will later be mounted by Batch AI. An alternative to uploading the scripts would be to embedd them inside the Docker container.

In [ ]:
!az storage file upload --share-name $FILE_SHARE_NAME --source ../src/imagenet_estimator_tf_horovod.py --path scripts
!az storage file upload --share-name $FILE_SHARE_NAME --source ../src/resnet_model.py --path scripts
!az storage file upload --share-name $FILE_SHARE_NAME --source ../common/timer.py --path scripts

Below it the command to create the cluster.

In [ ]:
!az batchai cluster create \
    -w $WORKSPACE \
    --name $CLUSTER_NAME \
    --image UbuntuLTS \
    --vm-size $VM_SIZE \
    --min $NUM_NODES --max $NUM_NODES \
    --afs-name $FILE_SHARE_NAME \
    --afs-mount-path extfs \
    --user-name $USERNAME \
    --password {get_password(dotenv_path)} \
    --storage-account-name $STORAGE_ACCOUNT_NAME \
    --storage-account-key $storage_account_key \
    --nfs $NFS_NAME \
    --nfs-mount-path nfs 

Let's check that the cluster was created succesfully.

In [ ]:
!az batchai cluster show -n $CLUSTER_NAME -w $WORKSPACE

In [ ]:
!az batchai cluster list -w $WORKSPACE -o table

In [ ]:
!az batchai cluster node list -c $CLUSTER_NAME -w $WORKSPACE -o table

<a id='job'></a>
## Submit and Monitor Job
Below we specify the job we wish to execute.  

In [ ]:
jobs_dict = {
  "$schema": "https://raw.githubusercontent.com/Azure/BatchAI/master/schemas/2017-09-01-preview/job.json",
  "properties": {
    "nodeCount": NUM_NODES,
    "customToolkitSettings": {
      "commandLine": f"source /opt/intel/compilers_and_libraries_2017.4.196/linux/mpi/intel64/bin/mpivars.sh; \
      echo $AZ_BATCH_HOST_LIST; \
      mpirun -n {TOTAL_PROCESSES} -ppn {PROCESSES_PER_NODE} -hosts $AZ_BATCH_HOST_LIST \
      -env I_MPI_FABRICS=dapl \
      -env I_MPI_DAPL_PROVIDER=ofa-v2-ib0 \
      -env I_MPI_DYNAMIC_CONNECTION=0 \
      -env I_MPI_DEBUG=6 \
      -env I_MPI_HYDRA_DEBUG=on \
      -env DISTRIBUTED=True \
      {FAKE} \
      python -u $AZ_BATCHAI_INPUT_SCRIPTS/imagenet_estimator_tf_horovod.py"
    },
    "stdOutErrPathPrefix": "$AZ_BATCHAI_MOUNT_ROOT/extfs",
    "inputDirectories": [{
        "id": "SCRIPTS",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/extfs/scripts"
      },
      {
        "id": "TRAIN",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/nfs/imagenet",
      },
      {
        "id": "TEST",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/nfs/imagenet",
      },
    ],
    "outputDirectories": [{
        "id": "MODEL",
        "pathPrefix": "$AZ_BATCHAI_MOUNT_ROOT/extfs",
        "pathSuffix": "Models"
    }],
    "containerSettings": {
      "imageSourceRegistry": {
        "image": f"{DOCKERHUB}/distributed-training.horovod-tf"
      }
    }
  }
}

In [ ]:
write_json_to_file(jobs_dict, 'job.json')

In [ ]:
JOB_NAME='tf-horovod-{}'.format(NUM_NODES*PROCESSES_PER_NODE)

We now submit the job to Batch AI

In [ ]:
!az batchai job create -n $JOB_NAME --cluster $CLUSTER_NAME -w $WORKSPACE -e $EXPERIMENT -f job.json

With the command below we can check the status of the job

In [ ]:
!az batchai job list -w $WORKSPACE -e $EXPERIMENT -o table

To view the files that the job has generated use the command below

In [ ]:
!az batchai job file list -w $WORKSPACE -e $EXPERIMENT --j $JOB_NAME --output-directory-id stdouterr

We are also able to stream the stdout and stderr that our job produces. This is great to check the progress of our job as well as debug issues.

In [ ]:
!az batchai job file stream -w $WORKSPACE -e $EXPERIMENT --j $JOB_NAME --output-directory-id stdouterr -f stdout.txt

In [ ]:
!az batchai job file stream -w $WORKSPACE -e $EXPERIMENT --j $JOB_NAME --output-directory-id stdouterr -f stderr.txt

We can either wait for the job to complete or delete it with the command below.

In [ ]:
!az batchai job delete -w $WORKSPACE -e $EXPERIMENT --name $JOB_NAME -y

<a id='clean_up'></a>
## Clean Up Resources
Next we wish to tidy up the resource we created.  
First we reset the default values we set earlier.

In [ ]:
!az configure --defaults group=''
!az configure --defaults location=''

 Next we delete the cluster

In [ ]:
!az batchai cluster delete -w $WORKSPACE --name $CLUSTER_NAME -g $GROUP_NAME -y

Once the cluster is deleted you will not incur any cost for the computation but you can still retain your experiments and workspace. If you wish to delete those as well execute the commands below.

In [ ]:
!az batchai experiment delete -w $WORKSPACE --name $EXPERIMENT -g $GROUP_NAME -y

In [ ]:
!az batchai workspace delete -n $WORKSPACE -g $GROUP_NAME -y

Finally we can delete the group and we will have deleted everything created for this tutorial.

In [ ]:
!az group delete --name $GROUP_NAME -y